<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS 17 Methodology </span> </p>
<p style="font-weight:bold;"> <span style="font-size: 24px"> Business Logic with Scopes  </span> </p>

Import scopes are divided in several notebooks:
- [Calculation of Identities](../Import/1ImportScope-Identities)
- [Calculation of Present Values](../Import/2ImportScope-PresentValue)
- Calculation of Actuals
- [Calculation of Technical Margin](../Import/4ImportScope-TechnicalMargin)
- [Creation of Ifrs Variables](../Import/5ImportScope-ToIfrsVar)
- [Calculation of Ifrs Variables](../Import/6ImportScope-Compute)

<br><br>
In this notebook the focus is on the calculation of actuals values.

# References
Libraries and other notebooks which are needed for this notebook are imported below.

## Notebooks

In [0]:
#!import "2ImportScope-PresentValue"

# Actual Values

We consider 4 types of Actual values, which are distinguished through their [Estimate Type](../DataModel/DataStructure#estimate-type):
- [Actuals](#actuals) (A)
- [Advance Actuals](#advance-actuals) (AA)
- [Overdue Actuals](#overdue-actuals) (OA)
- [Deferrable Actuals](#deferrable-actuals) (DA)

with the Estimate Type's system name shown between parenthesis above.

The following simplified AoC Chain applies for Advance and Overdue Actuals:
<br>&emsp;BOP
<br>&emsp;Release
<br>&emsp;Write-off
<br>&emsp;EOP

## Actual Base

The Actual Base sets values for actual, advance, and overdue as follows:

$$
\text{Actual Base} (\text{AoC step}) = \left\{
\begin{array}{rl}
0 ~, & \text{if AoC Step's AoC Type = AM} \\
\text{Actual Base}(\rm{BOP}) + \text{Actual Base}(\rm{CF}) + \text{Actual Base}(\rm{WO}) ~, & \text{if AoC Step's AoC Type = EOP and EstimateType is not A} \\
\text{Imported Actual} ~, & \text{otherwise}
\end{array}
\right.
$$

where the value is also function of the [Estimate Type](../DataModel/DataStructure#EstimateType) and [Amount Type](../DataModel/DataStructure#AmountType), and the $\text{Imported Actual}$ value is described [here]().

In [0]:
public interface ActualBase : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? AccidentYear), ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
            builder.ForScope<ActualBase>(s => s.WithApplicability<EmptyValuesActual>(x => x.GetStorage().ImportFormat == ImportFormats.Actual 
                                                                                       && !x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode)
                                                                                       && x.Identity.Id.AocType == AocTypes.AM)
                                               .WithApplicability<EndOfPeriodActual>(x => x.GetStorage().ImportFormat != ImportFormats.Cashflow 
                                                                                       && !x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode)
                                                                                       && x.Identity.Id.AocType == AocTypes.EOP 
                                                                                       && x.Identity.EstimateType != EstimateTypes.A)
                                       );
    public double Value => GetStorage().GetValue(Identity.Id, Identity.AmountType, Identity.EstimateType, Identity.AccidentYear, Identity.Id.ProjectionPeriod); 
}

public interface EndOfPeriodActual : ActualBase
{
    double ActualBase.Value => GetScope<PreviousAocSteps>((Identity.Id, InputSource.Actual)).Values
                                        .Sum(aocStep => GetScope<ActualBase>((Identity.Id with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}, Identity.AmountType, Identity.EstimateType, Identity.AccidentYear)).Value);
}

public interface EmptyValuesActual : ActualBase
{
    double ActualBase.Value => 0;
}

## Actuals
The Actuals correspond to ActualBase values with estimate type $A$.
The only valid AoC Step is Release:

$$
\text{Actual} (\text{Release}) = \text{Actual Base} (\text{Release})|_{\text{Estimate Type} = A}
$$

In [0]:
public interface Actual : IScope<ImportIdentity, ImportStorage>
{
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.A;
    
    private int?[] accidentYears => GetStorage().GetAccidentYears(Identity.DataNode).ToArray();
    
    [NotVisible]
    ActualBase[] Actuals => GetScope<ValidAmountType>(Identity.DataNode).ActualAmountTypes
                                                            .SelectMany(at => accidentYears
                                                                .Select(ay => GetScope<ActualBase>((Identity, at, EstimateType, ay)))).ToArray();
}

## Advance Actuals

Advance Actuals are cash flows with due date inside the reporting period but occured *before* the reporting period - They include:
- Receivable Claims
- Receivable Expenses
- Payable Premiums

Advance Actuals are given by

$$
\text{Advance Actual} (\text{AoC step}) = \text{Actual Base} (\text{AoC step})|_{\text{Estimate Type} = AA}
$$

In [0]:
public interface AdvanceActual : IScope<ImportIdentity, ImportStorage>
{
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.AA;
    
    private int?[] accidentYears => GetStorage().GetAccidentYears(Identity.DataNode).ToArray();
    
    [NotVisible]
    ActualBase[] Actuals => GetScope<ValidAmountType>(Identity.DataNode).ActualAmountTypes
                                                            .SelectMany(at => accidentYears
                                                                .Select(ay => GetScope<ActualBase>((Identity, at, EstimateType, ay)))).ToArray();
}

## Overdue Actuals

Overdue Actuals are cash flows with due date inside the reporting period but occur *after* the reporting period - They contain:
- Payable Claims
- Payable Expenses
- Receivable Premiums

The sign convention is the inverse of the default behavior - In particular: Premiums have positive value, whereas Claims and Expenses have negative value.

The Overdue Actuals are given by

$$
\text{Overdue Actual} (\text{AoC step}) = \text{Actual Base} (\text{AoC step})|_{\text{Estimate Type} = OA}
$$

In [0]:
public interface OverdueActual : IScope<ImportIdentity, ImportStorage>
{
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.OA;
    
    private int?[] accidentYears => GetStorage().GetAccidentYears(Identity.DataNode).ToArray();
    
    [NotVisible]
    ActualBase[] Actuals => GetScope<ValidAmountType>(Identity.DataNode).ActualAmountTypes
                                                            .SelectMany(at => accidentYears
                                                                .Select(ay => GetScope<ActualBase>((Identity, at, EstimateType, ay)))).ToArray();
}

## Deferrable Actuals

Finally, Deferrable Actuals are given by

$$
\text{Deferrable Actual} (\text{AoC step}) = \left\{
\begin{array}{rl}
\sum_{\text{Amount Type}~\in~\text{\{ACA, AEA}\}}\text{Actual Base}(\rm{CF})|_{\text{Estimate Type = A}} ~, & \text{if AoC Step's AoC Type = CF } \\
- \text{Amortization Factor} \cdot \big(  \text{Deferrable Actual}(\rm{BOP}) + \text{Deferrable Actual}(\rm{CF}) \big) ~, & \text{if AoC Step's AoC Type = AM } \\
\text{Deferrable Actual}(\rm{BOP}) + \text{Deferrable Actual}(\rm{CF}) + \text{Deferrable Actual}(\rm{AM}) ~, & \text{if AoC Step's AoC Type = EOP } \\
\text{Input Actual}|_{\text{Estimate Type = DA}} ~, & \text{ otherwise } \\
\end{array}
\right.
$$

where $ACA$ and $AEA$ are *Aquisition* Amount Types from **Attributable Commission** and **Attributable Expenses**, respectively.

In [0]:
public interface DiscountedAmortizationFactorForDeferrals : IScope<ImportIdentity, ImportStorage>
{
    private string EconomicBasis => GetContext();
    double Value => GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.DAE, 0), o => o.WithContext(EconomicBasis)).EffectiveAmountType == AmountTypes.DAE
        ? GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.DAE, 0), o => o.WithContext(EconomicBasis)).Value
        : GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.CU, 0), o => o.WithContext(EconomicBasis)).Value;
    string EffectiveAmountType => GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.DAE, 0), o => o.WithContext(EconomicBasis)).EffectiveAmountType == AmountTypes.DAE
        ? AmountTypes.DAE
        : GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.CU, 0), o => o.WithContext(EconomicBasis)).EffectiveAmountType;
}

In [0]:
public interface DeferrableActual : IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
            builder.ForScope<DeferrableActual>(s => s.WithApplicability<DeferrableActualForCurrentBasis>(x => x.Identity.ValuationApproach == ValuationApproaches.VFA, 
                                                                                                              p => p.ForMember(s => s.EconomicBasis))
                                                     .WithApplicability<ReleaseDeferrable>(x => x.Identity.AocType == AocTypes.CF)
                                                     .WithApplicability<AmortizationDeferrable>(x => x.Identity.AocType == AocTypes.AM)
                                                     .WithApplicability<EndOfPeriodDeferrable>(x => x.Identity.AocType == AocTypes.EOP)
                                       );
    
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.DA;
    [NotVisible] string EconomicBasis => EconomicBases.L;
    
    public double Value => GetStorage().GetValue(Identity, (string)null, EstimateType, (int?)null, Identity.ProjectionPeriod);
}

public interface DeferrableActualForCurrentBasis : DeferrableActual{
    [NotVisible] string DeferrableActual.EconomicBasis => EconomicBases.C;
}

public interface ReleaseDeferrable : DeferrableActual{
    double DeferrableActual.Value => GetStorage().GetAttributableExpenseAndCommissionAmountType().Sum(at => GetScope<ActualBase>((Identity, at, EstimateTypes.A, (int?)null)).Value);
}

public interface AmortizationDeferrable : DeferrableActual{
    private double AmortizationFactor => GetScope<DiscountedAmortizationFactorForDeferrals>(Identity, o => o.WithContext(EconomicBasis)).Value;
    private double AggregatedDeferrable => GetScope<PreviousAocSteps>((Identity, InputSource.Actual)).Values
                                            .Sum(aocStep => GetScope<DeferrableActual>(Identity with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}).Value);
    double DeferrableActual.Value => -1d * AggregatedDeferrable * AmortizationFactor;
}

public interface EndOfPeriodDeferrable : DeferrableActual{
    double DeferrableActual.Value => GetScope<PreviousAocSteps>((Identity, InputSource.Actual)).Values
                                        .Sum(aocStep => GetScope<DeferrableActual>(Identity with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}).Value);
}

# Deferrable

Deferral variables are computed here using cashflow inputs. We distinguish between two approaches depending on the EconomicBasisDriver defined in the [DataNodeParameter](../DataModel/DataStructure#data-node-parameters):
- L (Locked-in) or C (Current) : discounting is applied (cf. [Deferrable as cumulated discounted cash flow](#deferrable-as-cumulated-discounted-cash-flow)),
- N (Nominal) : discounting is not applied (cf. [Deferrable undiscounted](#deferrable-undiscounted)).



## Deferrable as cumulated discounted cash flow
The cumulated discounted cash flow (CDC) or Present Value of deferral amount types is considered within this approach. 
The AoC Chain coincides with the one defined for Present Values. In addition, an Amortization step is computed using a release pattern defined in input (in the [DataNodeParameter](../DataModel/DataStructure#data-node-parameters) or as cash flow). 


$$
\text{Deferrable} (\text{AoC step}) = \left\{
\begin{array}{rl}
0 ~, & \text{if AoC Step's AoC Type = CF } \\
\text{PV}(\text{CF})- \text{Actual Base}(\text{CF})|_{\text{Estimate Type = A}} ~, & \text{if AoC Step's AoC Type = EA } \\
- \text{Amortization Factor} \cdot \sum_{\text{Previous AoC Step}} \text{Deferrable}(\text{AoC Step})  ~, & \text{if AoC Step's AoC Type = AM } \\
\sum_{\text{Previous AoC Step}} \text{Deferrable}(\text{AoC Step}) ~, & \text{if AoC Step's AoC Type = EOP } \\
\text{PV}(\text{AoC Step}) ~, & \text{ otherwise } \\
\end{array}
\right.
$$

In [0]:
public interface DiscountedDeferrable : IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<DiscountedDeferrable>(s => s //compute with IfrsVariable?
                                                 //.WithApplicability<BoPDeferrableProjection>(x => x.Identity.AocType == AocTypes.BOP && x.Identity.Novelty == Novelties.I && x.Identity.ProjectionPeriod > 0)
                                                 .WithApplicability<DeferrableForBOP>(x => x.Identity.AocType == AocTypes.BOP && x.Identity.Novelty == Novelties.I)
                                                 .WithApplicability<DeferrableDefaultValue>(x => x.Identity.AocType == AocTypes.CF)
                                                 .WithApplicability<DeferrableEa>(x => x.Identity.AocType == AocTypes.EA)
                                                 .WithApplicability<DeferrableAm>(x => x.Identity.AocType == AocTypes.AM)
                                                 .WithApplicability<DeferrableEoP>(x => x.Identity.AocType == AocTypes.EOP)
                                   );
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.DA;
    string AmountType => null;

    [NotVisible] string EconomicBasis => GetStorage().GetEconomicBasisDriver(Identity.DataNode);
    
    double Value => GetStorage().GetDeferrableExpenses().Sum(at => 
        GetScope<PresentValue>((Identity, at, EstimateTypes.BE, (int?)null), o => o.WithContext(EconomicBasis)).Value);
}

public interface DeferrableForBOP : DiscountedDeferrable {
    double DiscountedDeferrable.Value => GetStorage().GetValue(Identity, null, EstimateTypes.DA, EconomicBasis, (int?)null, Identity.ProjectionPeriod);
}

public interface DeferrableDefaultValue : DiscountedDeferrable {
    double DiscountedDeferrable.Value => default;
}

public interface DeferrableEa : DiscountedDeferrable {
    private string referenceAocType => GetScope<ReferenceAocStep>(Identity).Value.AocType;
    double DiscountedDeferrable.Value => GetStorage().GetDeferrableExpenses().Sum(at =>
        GetStorage().GetNovelties(referenceAocType, InputSource.Cashflow)
        .Sum(n => GetScope<PresentValue>((Identity with {AocType = referenceAocType, Novelty = n}, at, EstimateTypes.BE, (int?)null), o => o.WithContext(EconomicBasis)).Value) -
             GetScope<ActualBase>((Identity with {AocType = referenceAocType, Novelty = Novelties.C}, at, EstimateTypes.A, (int?)null)).Value);
}

public interface DeferrableAm : DiscountedDeferrable {
    private double AmortizationFactor => GetScope<DiscountedAmortizationFactorForDeferrals>(Identity, o => o.WithContext(EconomicBasis)).Value;
    private double AggregatedValue => GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow)).Values
                                            .Sum(aocStep => GetScope<DiscountedDeferrable>(Identity with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}).Value);
    double DiscountedDeferrable.Value => -1d * AggregatedValue * AmortizationFactor;
}

public interface DeferrableEoP : DiscountedDeferrable {
    double DiscountedDeferrable.Value => GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow)).Values
                                        .Sum(aocStep => GetScope<DiscountedDeferrable>(Identity with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}).Value);
}

## Deferrable undiscounted
The nominal cash flow of deferral amount types is considered within this approach. 
As Present Values are not used here, this methodology allows to amortize and release expenses occurring in different months within the reporting period. 
This is achieved by applying a shift to the release pattern. For this reason, this approach is to be used with a release pattern defined in the [DataNodeParameter](../DataModel/DataStructure#data-node-parameters).

The AoC Chain is simplified and considers: 
- BOP (I as opening value to be entered as cash flow, N for new expenses)
- AM,
- EOP.


$$
\text{Deferrable} (\text{AoC step}) = \left\{
\begin{array}{rl}

\text{Previous Period EOP} ~, & \text{if AoC Step's AoC Type = BOP } \\
- \text{Amortization Factor}(shift) \cdot \sum_{\text{Previous AoC Step}} \text{Deferrable}(\text{AoC Step})  ~, & \text{if AoC Step's AoC Type = AM } \\
\sum_{\text{Previous AoC Step}} \text{Deferrable}(\text{AoC Step}) ~, & \text{if AoC Step's AoC Type = EOP } \\
0 ~, & \text{ otherwise } \\
\end{array}
\right.
$$

where the $shift$ corresponds to the month at which the cost occurs (starting from 0).



In [0]:
//TODO : 
// EstimateType from DA to DAC
// BOP,I only through Opening. 
public interface NominalDeferrable : IScope<(ImportIdentity Id, int MonthlyShift), ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
            builder.ForScope<NominalDeferrable>(s => s
                                                     .WithApplicability<BoPDeferrableProjection>(x => x.Identity.Id.AocType == AocTypes.BOP && x.Identity.Id.Novelty == Novelties.I && x.Identity.Id.ProjectionPeriod > 0)
                                                     .WithApplicability<BoPDeferrable>(x => x.Identity.Id.AocType == AocTypes.BOP)
                                                     .WithApplicability<ClDeferrable>(x => x.Identity.Id.AocType == AocTypes.CL)
                                                     .WithApplicability<AmDeferrable>(x => x.Identity.Id.AocType == AocTypes.AM)
                                                     .WithApplicability<EopDeferrable>(x => x.Identity.Id.AocType == AocTypes.EOP)
                                       );
    
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.DA;
    string AmountType => null;
    
    [NotVisible] string EconomicBasis => EconomicBases.N;
    double Value => default;
}

public interface BoPDeferrableProjection : NominalDeferrable{
    double NominalDeferrable.Value => GetScope<NominalDeferrable>((
        Identity.Id with {AocType = AocTypes.EOP, Novelty = Novelties.C, ProjectionPeriod = Identity.Id.ProjectionPeriod - 1}, 
        Identity.MonthlyShift)).Value;
}

public interface BoPDeferrable : NominalDeferrable{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
            builder.ForScope<NominalDeferrable>(s => s.WithApplicability<NominalDeferrableFromIfrsVariable>(x =>  x.GetStorage().ImportFormat != ImportFormats.Cashflow
                                                                                                 || x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode)
                                                                                                 || (x.Identity.Id.Novelty == Novelties.I && !x.GetStorage().IsInceptionYear(x.Identity.Id.DataNode)) ));
    private int projectionShift => GetStorage().GetShift(Identity.Id.ProjectionPeriod);
    double NominalDeferrable.Value => GetScope<NominalCashflow>((Identity.Id, AmountTypes.DAE, EstimateTypes.BE, (int?)null)).Values
        .Skip(projectionShift + Identity.MonthlyShift).FirstOrDefault();
}

public interface NominalDeferrableFromIfrsVariable : NominalDeferrable{
    double NominalDeferrable.Value => GetStorage().GetValue(Identity.Id, null, EstimateTypes.DA, EconomicBasis, Identity.MonthlyShift, Identity.Id.ProjectionPeriod);
}

public interface ClDeferrable : NominalDeferrable{
    private int projectionShift => GetStorage().GetShift(Identity.Id.ProjectionPeriod);
    private IEnumerable<AocStep> previousAocSteps => GetScope<PreviousAocSteps>((Identity.Id, InputSource.Cashflow)).Values;
    double referenceCashflow => previousAocSteps
        .GroupBy(x => x.Novelty, (k, aocs) => aocs.Last())
        .Sum(aoc => GetScope<NominalCashflow>((Identity.Id with {AocType = aoc.AocType, Novelty = aoc.Novelty}, AmountTypes.DAE, EstimateTypes.BE, (int?)null)).Values
        .Skip(projectionShift + Identity.MonthlyShift).FirstOrDefault());
     
    double NominalDeferrable.Value => referenceCashflow >= Precision //if no previous RawVariable, use IfrsVariable
        ? referenceCashflow 
        : GetScope<NominalDeferrable>((Identity.Id with {AocType = AocTypes.BOP, Novelty = Novelties.I}, Identity.MonthlyShift)).Value +
          GetScope<NominalDeferrable>((Identity.Id with {AocType = AocTypes.BOP, Novelty = Novelties.N}, Identity.MonthlyShift)).Value;
}

public interface AmDeferrable : NominalDeferrable{
    private AocStep referenceAocStep => GetScope<ReferenceAocStep>(Identity.Id).Value; //CL,C
    private double referenceCashflow => GetScope<NominalDeferrable>((Identity.Id with {AocType = referenceAocStep.AocType, Novelty = referenceAocStep.Novelty}, Identity.MonthlyShift)).Value;

    double NominalDeferrable.Value => -1d * referenceCashflow * GetScope<CurrentPeriodAmortizationFactor>((Identity.Id, AmountTypes.DAE, Identity.MonthlyShift), o => o.WithContext(EconomicBasis)).Value;
}

public interface EopDeferrable : NominalDeferrable{
    private IEnumerable<AocStep> previousAocSteps => GetScope<PreviousAocSteps>((Identity.Id, InputSource.Cashflow)).Values;
    double NominalDeferrable.Value => previousAocSteps.Sum(aocStep => GetScope<NominalDeferrable>((Identity.Id with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}, Identity.MonthlyShift)).Value);
}